In [ ]:
from buffer_integrity.prefect.tasks import ravg

import geopandas
import fsspec

import json

import rioxarray

import matplotlib.pyplot as plt

import pandas as pd

from carbonplan_styles.mpl.colormaps import fire_light
from carbonplan_styles.colors import light

# indirectly from matplotlib
from matplotlib.cm import get_cmap

fire_cmap = get_cmap("fire_light")

plt.rc("font", **{"size": 16})

In [ ]:
import numpy as np
import matplotlib
from matplotlib.colors import BoundaryNorm

import matplotlib

norm = matplotlib.colors.Normalize(vmin=0, vmax=tp_da.max())


shifted_cmap = matplotlib.colors.ListedColormap(
    [fire_cmap(norm(x)) for x in np.linspace(1, 7, 7)]
)

In [ ]:
da = ravg.load_ravg.run("lionshead")
subset = ravg.get_ravg_subset.run(da, "ACR260")
acres_burned = ravg.get_ravg_counts.run(subset)

In [ ]:
rpj = da.rio.reproject("epsg:4326")

In [ ]:
with fsspec.open(
    "https://storage.googleapis.com/carbonplan-forest-offsets/carb-geometries/raw/ACR260.json"
) as f:
    d = json.load(f)

shp = geopandas.GeoDataFrame.from_features(d)

shp = shp.set_crs("epsg:4326")

shp.geometry = shp.simplify(0.0001)

In [ ]:
tp_da = rpj.where((rpj > 0) & (rpj < 8))

In [ ]:
hist_data = pd.Series(acres_burned)


tp_hist = (hist_data / 1_000).sort_index()

tp_hist.index = tp_hist.index.astype(int)

In [ ]:
f, ax = plt.subplots(
    1, 2, gridspec_kw={"width_ratios": [3, 1]}, figsize=(7, 3.5)
)
orig_aspect = ax[0].get_aspect()

# plt.subplot2grid(shape=(3,3))
tp_da.plot(add_colorbar=False, ax=ax[0], cmap=shifted_cmap, rasterized=True)

shp.plot(ax=ax[0], color=light["secondary"], alpha=0.85)
ax[0].set_aspect(orig_aspect)
ax[0].set_title("")
# ax[0].spines['top'].set_visible(False)
# ax[0].spines['right'].set_visible(False)

# ax[0].xaxis.set_ticks_position('bottom')
# plt.axis('off')
for _, spine in ax[0].spines.items():
    spine.set_visible(False)
# ax[0].axis('off')
ax[0].grid()

ax[0].xaxis.set_ticks_position("none")
ax[0].yaxis.set_ticks_position("none")


ax[0].set_xticks([-121.5, -121, -120.5])
ax[0].set_xlim(-121.5 - 0.05, -120.5 + 0.05)
ax[0].set_xticklabels([])

ax[0].set_yticks([42, 42.5, 43])
ax[0].set_ylim(42 - 0.05, 43 + 0.05)
ax[0].set_yticklabels([])

ax[0].set_ylabel(None)
ax[0].set_xlabel(None)


tp_hist.plot.bar(ax=ax[1], color=shifted_cmap.colors, grid=False)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=0)

ax[1].spines["top"].set_visible(False)
ax[1].spines["right"].set_visible(False)
ax[1].set_ylabel("Project Area\n(thousand acres)")
ax[1].set_xlabel("Mortality Class", labelpad=10)
ax[1].set_yticks([0, 25, 50])
ax[1].set_ylim(0, 50)

cb_ax = ax[0].inset_axes([0.3, -0.10, 0.45, 0.05])

bounds = np.linspace(1, 8, 8)
norm = BoundaryNorm(bounds, shifted_cmap.N)
cb = matplotlib.colorbar.ColorbarBase(
    cb_ax,
    cmap=shifted_cmap,
    norm=norm,
    spacing="proportional",
    orientation="horizontal",
    boundaries=bounds,
)
cb.set_ticks([])
cb.set_label("Mortality Class", labelpad=12.5)
cb.ax.text(-0.1, 1, "1", transform=cb.ax.transAxes, va="top", ha="center")
cb.ax.text(1.1, 1, "7", transform=cb.ax.transAxes, va="top", ha="center")


plt.subplots_adjust(wspace=0.5)
plt.savefig("../img/bootleg.pdf", bbox_inches="tight", dpi=300)

In [ ]:
f, ax = plt.subplots(
    1, 2, gridspec_kw={"width_ratios": [3, 1]}, figsize=(7, 3.5)
)
orig_aspect = ax[0].get_aspect()

# plt.subplot2grid(shape=(3,3))
tp_da.plot(add_colorbar=False, ax=ax[0], cmap=shifted_cmap, rasterized=True)

shp.plot(ax=ax[0], color=light["secondary"], alpha=0.85)
ax[0].set_aspect(orig_aspect)
ax[0].set_title("")
# ax[0].spines['top'].set_visible(False)
# ax[0].spines['right'].set_visible(False)

# ax[0].xaxis.set_ticks_position('bottom')
# plt.axis('off')
for _, spine in ax[0].spines.items():
    spine.set_visible(False)
# ax[0].axis('off')
ax[0].grid()

ax[0].xaxis.set_ticks_position("none")
ax[0].yaxis.set_ticks_position("none")


ax[0].set_xticks([-122.5, -122, -121.5])
ax[0].set_xlim([-122.5 - 0.05, -121.5 + 0.05])
ax[0].set_xticklabels([])

ax[0].set_yticks([44.5, 44.75, 45])
ax[0].set_ylim([44.5 - 0.05, 45 + 0.05])
ax[0].set_yticklabels([])

# ax[0].spines['bottom'].set_position(('outward', 10))
# ax[0].spines['left'].set_position(('outward', 10))
# ax[0].set_yticks([42, 42.5, 43])
# ax[0].set_ylim(42,43)
ax[0].set_ylabel(None)
ax[0].set_xlabel(None)

# ax[0].set_xticks([-121.5, -121, -120.5])
# ax[0].set_xlim(-121.5, -120.5)


# cb.ax.tick_params(size=0)
tp_hist.plot.bar(ax=ax[1], color=shifted_cmap.colors, grid=False)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=0)

ax[1].spines["top"].set_visible(False)
ax[1].spines["right"].set_visible(False)
ax[1].set_ylabel("Project Area\n(thousand acres)")
ax[1].set_xlabel("Mortality Class", labelpad=10)
# ax[1].set_yticks([0,25,50])
ax[1].set_ylim(0, 8)

cb_ax = ax[0].inset_axes([0.3, -0.10, 0.45, 0.05])

bounds = np.linspace(1, 8, 8)
norm = BoundaryNorm(bounds, shifted_cmap.N)
cb = matplotlib.colorbar.ColorbarBase(
    cb_ax,
    cmap=shifted_cmap,
    norm=norm,
    spacing="proportional",
    orientation="horizontal",
    boundaries=bounds,
)
cb.set_ticks([])
cb.set_label("Mortality Class", labelpad=12.5)
cb.ax.text(-0.1, 1, "1", transform=cb.ax.transAxes, va="top", ha="center")
cb.ax.text(1.1, 1, "7", transform=cb.ax.transAxes, va="top", ha="center")

# cb_ax.colorbar(orientation='horizontal')

plt.subplots_adjust(wspace=0.5)
plt.savefig("../img/lionshead.pdf", bbox_inches="tight", dpi=300)